In [ ]:
import time
import warnings

import matplotlib as mpl
import numpy as np
from IPython import get_ipython

import pymor.tools.random

ip = get_ipython()
if ip is not None:
#    ip.run_line_magic('matplotlib', 'inline')
    ip.run_line_magic('matplotlib', 'ipympl')

warnings.filterwarnings('ignore', category=UserWarning, module='torch')

pymor.tools.random._default_random_state = None

mpl.rcParams['figure.facecolor'] = (1.0, 1.0, 1.0, 0.0)

from pymor.core.logger import getLogger, set_log_levels

set_log_levels({ # disable logging for some components
    'main': 'DEBUG',
    'pymor': 'WARN',
    'pymor.models': 'WARN',
    'pymor.discretizers.builtin': 'WARN',
    'pymor.discretizers.dunegdt': 'DEBUG',
    'pymor.analyticalproblems.functions.BitmapFunction': 'ERROR',
    'models.ann.ANNStateReductor': 'INFO',
    'models.vkoga.VkogaStateModel': 'INFO',
    'models.vkoga.VkogaStateReductor': 'DEBUG',
    'models.adaptive': 'DEBUG',
    'algorithms.optimization': 'DEBUG'})

logger = getLogger('main.main')

In [ ]:
# FOM
num_refines = 0
num_timesteps = 10

In [ ]:
from pymor.basic import *


def setup_problem_and_discretize(num_refines, nt):
    from spe10channel import discretize, make_problem

    grid, boundary_info, problem, parameter_space, mu_bar = make_problem(
        regime='diffusion dominated', num_global_refines=num_refines)

    fom, fom_data, coercivity_estimator = discretize(
        grid, boundary_info, problem, mu_bar, nt=nt)

    return parameter_space, fom, fom_data, coercivity_estimator


spatial_product = lambda m: m.energy_0_product

In [ ]:
logger.info('creating FOM:')
tic = time.perf_counter()

parameter_space, fom, fom_data, coercivity_estimator = setup_problem_and_discretize(
    num_refines, num_timesteps)

fom_offline_time = time.perf_counter() - tic
logger.info(f'  discretizing took {fom_offline_time}s')
logger.info(f'  grid has {fom_data["grid"].size(0)} elements,'
            f'FOM has {fom.solution_space.dim} DoFs, '
            f'uses {fom.time_stepper.nt} time steps')

logger.info(f'  input parameter space is {parameter_space.parameters.dim}-dimensional:')
logger.info(f'    {parameter_space}')

In [ ]:
logger.info('computing dual norm of output functional:')

assert not fom.output_functional.parametric
riesz_representative = spatial_product(fom).apply_inverse(fom.output_functional.as_vector())
dual_norm_output = np.sqrt(spatial_product(fom).apply2(riesz_representative, riesz_representative)[0][0])
del riesz_representative

logger.info(f'  {dual_norm_output}')

In [ ]:
mu_ref = parameter_space.sample_uniformly(1)[0]

logger.info(f'computing f_h(mu={mu_ref}) ...')

tic = timer()
f_mu_ref = fom.output(mu=mu_ref, incremental=True)  # no need to keep the state trajectory in memory
fom_online_output_time = timer() - tic

logger.info(f'average FOM output (solve + apply functional) time: {fom_online_output_time}s')

logger.info('computing ||f_mu_ref||_{L^2(0, T)}:')
initial_abs_output_tol = fom.output_l2_norm(f_mu_ref)
logger.info(f'  {initial_abs_output_tol}')

In [ ]:
from pymor.models.hierarchy import AdaptiveModelHierarchy
from pymor.reductors.neural_network import NeuralNetworkReductor

In [ ]:
rb_reductor = CoerciveRBReductor(
    fom,
    product=fom.h1_0_semi_product,
    coercivity_estimator=ExpressionParameterFunctional('min(diffusion)', fom.parameters)
)
#ParabolicRBReductor, product=spatial_product(fom), coercivity_estimator=coercivity_estimator

def reduction_rb(training_data, len_previous_training_data, models, reductors):
    U = fom.solution_space.empty(reserve=len(training_data))
    for _, u in training_data[len_previous_training_data:]:
        U.append(u)
    RB, _ = pod(U, product=fom.h1_0_semi_product)
    reductors[0].extend_basis(RB)
    return reductors[0].reduce()

def post_reduction_rb(training_data, models, reductors):
    return []

tolerance = 5e-3

# Settings for the two-stage hierarchy
models = [rb_reductor.reduce(), fom]
model_names = ['RB-ROM', 'FOM']
reductors = [rb_reductor]
reduction_methods = [reduction_rb]
post_reduction_methods = [post_reduction_rb]
training_frequencies = [1]

two_stage_hierarchy = AdaptiveModelHierarchy(models, reductors, reduction_methods, post_reduction_methods,
                                             training_frequencies, tolerance, visualizer=fom.visualizer,
                                             name='Two-stage model hierarchy')

In [ ]:
def Linf_misfit(m, x, f_ref=None, print_status=False):
    if print_status:
        print('.', end='', flush=True)
    mu = m.parameters.parse(x)
    f_x = m.output(mu, incremental=True)
    if f_ref is None:
        f_ref = m.output(mu=mu_ref, incremental=True)
    return np.max(np.abs((f_ref - f_x).to_numpy()))

In [ ]:
from pymor.models.interact import interact_model_hierarchy

In [ ]:
interact_model_hierarchy(two_stage_hierarchy, parameter_space, model_names,
                         partial(Linf_misfit, f_ref=f_mu_ref))

In [ ]:
ml_reductor = NeuralNetworkReductor(fom=fom, training_set=None, validation_set=None,
                                    ann_mse=None, pod_params={'product': fom.h1_0_semi_product},
                                    scale_inputs=False, scale_outputs=False)

def reduction_ml(training_data, len_previous_training_data, models, reductors):
    rb_rom = models[1]
    rb_reductor = reductors[1]
    ml_reductor = reductors[0]
    error_estimator = rb_rom.error_estimator
    ml_reductor.reduced_basis = rb_reductor.bases['RB']
    red_dim = len(ml_reductor.reduced_basis)
    training_data = [(mu, np.pad(dat.to_numpy()[0], (0, red_dim - len(dat.to_numpy()[0])),
                                 mode='constant', constant_values=0.))
                     for (mu, dat) in training_data]
    ml_reductor.training_data = training_data
    ml_rom = ml_reductor.reduce(restarts=2, log_loss_frequency=10, recompute_training_data=False,
                                recompute_validation_data=True)
    return ml_rom.with_(error_estimator=error_estimator)

def post_reduction_ml(training_data, models, reductors):
    return []

def post_reduction_rb(training_data, models, reductors):
    return [models[0]]
    #return [reduction_ml(training_data[1], None, models, reductors)]

models = [None, rb_reductor.reduce(), fom]
model_names = ['ML-ROM', 'RB-ROM', 'FOM']
reductors = [ml_reductor, rb_reductor]
reduction_methods = [reduction_ml, reduction_rb]
post_reduction_methods = [post_reduction_ml, post_reduction_rb]
training_frequencies = [20, 1]

three_stage_hierarchy = AdaptiveModelHierarchy(models, reductors, reduction_methods, post_reduction_methods,
                                               training_frequencies, tolerance)

In [ ]:
interact_model_hierarchy(three_stage_hierarchy, parameter_space, model_names,
                         visualizer=fom.visualize)